Tema do Projeto: Sistema de recomendação de filmes

Autor: Gustavo Diniz Monteiro

Este projeto tem como objetivo desenvolver um sistema de recomendação de filmes. Baseando-se nos dados retirados do site Movielens. A partir desses dados foi desenvolvido um sistema de recomendação de filmes baseado em dois algoritmos (KNN e SVD) que serão mostrados abaixo.

In [2]:
import os
import pandas as pd
from surprise import Dataset, KNNBasic, Reader, accuracy, SVD
from surprise.model_selection import cross_validate, PredefinedKFold

Carregando os dados

Abaixo os dados são lidos de modo a criar um dataset do conjunto de treino e de teste,
um set de ids de usuários e um set de ids de items (filmes). O conjunto de dados utilizado é o ml-100k

In [3]:
items_stream = open('ml-100k/u.item', 'r')
item_data = items_stream.read().split('\n')
item_data = list(map(lambda item: item.split('|')[:2], item_data))
items_stream.close()

In [10]:
database = pd.read_csv('ml-100k/data.csv')
user_set = set(database.user_id)
item_set = set(database.item_id)
not_watch = {user: item_set - set(database.query('user_id == %s' %(user)).item_id) for user in user_set}

In [11]:
files_dir = os.path.expanduser('ml-100k/')
reader = Reader('ml-100k')

train_file = files_dir + 'u1.base'
test_file = files_dir + 'u1.test'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)
pkf = PredefinedKFold()

In [12]:
sim_options = {
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
}

algo_knn = KNNBasic(sim_options=sim_options, k=4, min_k=2)
algo_svd = SVD()

for trainset, testset in pkf.split(data):

    # train and test algorithm.
    algo_knn.fit(trainset)
    algo_svd.fit(trainset)
    predictions_knn = algo_knn.test(testset)
    predictions_svd = algo_svd.test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [13]:
def get_top_5_knn(uid):
    top = []
    items = not_watch[int(uid)]
    
    for item in items:
        top.append((item, algo_knn.predict(uid=uid, iid=str(item)).est))
    
    return sorted(top, key=lambda item: item[1], reverse=True)[:5]


def get_top_5_movies_knn(uid):
    top_5 = get_top_5_knn(uid)
    return [item_data[int(item[0])][1] for item in top_5]

In [14]:
def get_top_5_svd(uid):
    top = []
    items = not_watch[int(uid)]
    
    for item in items:
        top.append((item, algo_svd.predict(uid=uid, iid=str(item)).est))
    
    return sorted(top, key=lambda item: item[1], reverse=True)[:5]


def get_top_5_movies_svd(uid):
    top_5 = get_top_5_svd(uid)
    return [item_data[int(item[0])][1] for item in top_5]

In [15]:
def get_top_5_neighbors(uid):
    inner_uid = algo_knn.trainset.to_inner_uid(uid)
    neighbords = algo_knn.get_neighbors(iid=inner_uid, k=5)
    return [algo_knn.trainset.to_raw_uid(iid) for iid in neighbords]